In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from arcgis.geometry import Geometry, SpatialReference, Point, Polygon, buffer
from getpass import getpass
from arcgis.features import FeatureLayer, Feature, FeatureLayerCollection
import pandas as pd
from datetime import datetime

In [ ]:
# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
username = " "
gis = GIS("https://www.arcgis.com", username, getpass())

In [ ]:
find_storymaps = gis.content.search("title: Community Storymap", item_type="Web Mapping Application", max_items=500, outside_org=False)


In [ ]:
#Set up a log file to write errors to
#today = [dt.date.today()]
#log_path = r"T:\CDO_To_Esri\Storymaps\_Archive\Logfiles\Update General Services"
#log_file = os.path.join(log_path, "Update General Services_" + str(today[0]) + str(".txt"))
#out = open(log_file,'w')

for f in find_storymaps:
    try:
        if f.title != 'Alaska DCRA Community Storymaps' and not f.title.startswith('Communit') and 'Community' in str(f.title):
            csmap = storymap.JournalStoryMap(gis.content.get(str(f.id)))
            sections = csmap.properties["values"]["story"]["sections"]
            community = f.title.replace(" Community Storymap", "").replace(" (Unincorporated Community)", "").replace("'","''")
            print(f.title, f.id)
            if community.startswith("City & Borough of "):
                community = community.replace("City & Borough of ", "")
            if community.startswith("Municipality of "):
                community = community.replace("Municipality of ","")
            #Define the general services panel and change the main content
            panel_contacts = sections[11]
            
            ###ORIGINAL BUFFERING PROCESS TO DEFINE WEB MAP EXTENT
            #reg_lyr = FeatureLayer('https://maps.commerce.alaska.gov/server/rest/services/Hosted/CDO_Community_Regions_Overview/FeatureServer/0')
            #com_query = reg_lyr.query(where="communityname = '" + str(community) + "'")
            #c_ft = Geometry(com_query.features[0].geometry)
            #c_pt = Point({"x": c_ft.x, "y": c_ft.y, "spatialReference": {"wkid": 102006}})
            #print("Feature point: ", c_pt,  type(c_pt))
            
            #base_ext = c_pt.buffer(4000).extent
            #c_ext = Geometry({
            #    'xmin':base_ext[0],
            #    'ymin':base_ext[1],
            #    'xmax':base_ext[2],
            #    'ymax':base_ext[3],
            #    'spatialReference':{
            #        'wkid':102006
            #    }
            #}).project_as(102100)
            #print("Extent: ", c_ext)
            
            ###NEW BUFFERING METHOD
            community_base = FeatureLayer('https://services2.arcgis.com/0DjevcWawQ1dy3il/arcgis/rest/services/Community_Index/FeatureServer/0')
            
            basept_lyr = community_base.query(where="CommunityN = '" + str(community) + "'", out_fields = "*")
            c_ft = basept_lyr.features[0]
            gbuf = buffer(geometries=[Geometry(c_ft.geometry)], in_sr=102966, distances=[8000], unit='Meters', out_sr=102100) # assume meters, 1609.344 ~ 1 mile 

            geom = Geometry({
                "rings" : gbuf[0]["rings"],
                "spatialReference" : {"wkid" : 102100}
            })

            base_ext=geom.extent
            c_ext = Geometry({
                'xmin':base_ext[0],
                'ymin':base_ext[1],
                'xmax':base_ext[2],
                'ymax':base_ext[3],
                'spatialReference' : {'wkid':102100}
            })
            
            if 'Contacts' in str(panel_contacts['title']):
                print(panel_contacts["title"])
                contacts_wm = WebMap(webmapitem=gis.content.get("c68f6b0d441a4949b17e0ac3b98774a8"))

                lyr_lst = []
                for lyr in contacts_wm.layers:
                    lyr_lst.append(
                    {
                        "id": lyr.id,
                        "visibility":True
                    })

                panel_contacts["media"] ={}
                print("Media panel cleared.")
                panel_contacts["media"] = {
                            "type": "webmap",
                            "webmap": {
                                "id": "c68f6b0d441a4949b17e0ac3b98774a8",
                                "extent": {
                                    "xmin": c_ext.xmin,
                                    "ymin": c_ext.ymin,
                                    "xmax": c_ext.xmax,
                                    "ymax": c_ext.ymax,
                                    "spatialReference": {
                                        "wkid": 102100
                                    }
                                },
                                "layers": lyr_lst,
                                "popup": None,
                                "overview": {
                                    "enable": False,
                                    "openByDefault": False
                                },
                                "legend": {
                                    "enable": True,
                                    "openByDefault": True
                                },
                                "geocoder": {
                                    "enable": False
                                },
                                "altText": "Locations schools, safety net healthcare facilities, and the community. Icons can be clicked to access contact information for each item."
                            }
                        }


                csmap.save()
                print("Contacts map updated.")
    except Exception as e:
        print("-------ISSUE WITH" + str(f.title) + "--------" + str(e))
        #out.write('Unable to update demographics map for ' + str(f.title) + ". " + str(e))
        #out.write('\n\n')
        continue

In [ ]:
#Create a panel extent definition 

community = 'Angoon'
community_base = FeatureLayer('https://maps.commerce.alaska.gov/arcgis/rest/services/Community_Related/Community_Overview/MapServer/0')

basept_lyr = community_base.query(where="CommunityName = '" + str(community) + "'", out_fields = "*")

c_ft = basept_lyr.features[0]
gbuf = buffer(geometries=[Geometry(c_ft.geometry)], in_sr=4326, distances=[8800], unit='Meters', out_sr=102100) # assume meters, 1609.344 ~ 1 mile 

geom = Geometry({
    "rings" : gbuf[0]["rings"],
    "spatialReference" : {"wkid" : 102100}
})

base_ext=geom.extent
c_ext = Geometry({
    'xmin':base_ext[0],
    'ymin':base_ext[1],
    'xmax':base_ext[2],
    'ymax':base_ext[3],
    'spatialReference' : {'wkid':102100}
})



In [ ]:
panel_contacts.title